In [ ]:
import numpy as np
import random
import tensorflow as tf
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
class Network(object):
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                correct_test = self.evaluate(test_data)
                correct_train = self.evaluate_train(training_data)
                print("Epoch {0}: Val {1}/{2} - Accuracy: {3}, Train {4}/{5} - Accuracy: {6}".format(
                    j, correct_test, n_test, round(correct_test / n_test, 3), 
                    correct_train, len(training_data), round(correct_train / len(training_data), 3)
                ))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]
    # BACKPROPAGATION
    
    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        activation = x
        activations = [x]
        zs = [] 
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta.reshape(-1, 1), activations[-2].reshape(-1, 1).transpose())
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta.reshape(-1, 1), activations[-l-1].reshape(-1, 1).transpose())
        return (nabla_b, nabla_w)

# ACTIVATION FUNCTIONS: 

def sigmoid(z):
    return 1/(1 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [ ]:
data=load_breast_cancer()


In [ ]:
df = pd.DataFrame(data.data,columns=data.feature_names)
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(data.data,data.target,test_size=0.33)


In [ ]:
scale=StandardScaler()
x_train=scale.fit_transform(x_train)
x_test=scale.transform(x_test)

In [ ]:
N,D=x_train.shape
print(N,D)

381 30


In [ ]:

training_data=list(zip(x_train,y_train))
test_data=list(zip(x_test,y_test))

In [ ]:
net = Network([D, 16, 2])
net.SGD(training_data, 100, 64, 2, test_data=test_data)
# SGD(training_data,epochs,batchsize,eta,testdata)

Epoch 0: Val 106/188 - Accuracy: 0.564, Train 216/381 - Accuracy: 0.567
Epoch 1: Val 105/188 - Accuracy: 0.559, Train 196/381 - Accuracy: 0.514
Epoch 2: Val 105/188 - Accuracy: 0.559, Train 198/381 - Accuracy: 0.52
Epoch 3: Val 104/188 - Accuracy: 0.553, Train 208/381 - Accuracy: 0.546
Epoch 4: Val 103/188 - Accuracy: 0.548, Train 210/381 - Accuracy: 0.551
Epoch 5: Val 105/188 - Accuracy: 0.559, Train 212/381 - Accuracy: 0.556
Epoch 6: Val 106/188 - Accuracy: 0.564, Train 214/381 - Accuracy: 0.562
Epoch 7: Val 109/188 - Accuracy: 0.58, Train 223/381 - Accuracy: 0.585
Epoch 8: Val 111/188 - Accuracy: 0.59, Train 223/381 - Accuracy: 0.585
Epoch 9: Val 113/188 - Accuracy: 0.601, Train 221/381 - Accuracy: 0.58
Epoch 10: Val 114/188 - Accuracy: 0.606, Train 221/381 - Accuracy: 0.58
Epoch 11: Val 116/188 - Accuracy: 0.617, Train 220/381 - Accuracy: 0.577
Epoch 12: Val 116/188 - Accuracy: 0.617, Train 223/381 - Accuracy: 0.585
Epoch 13: Val 115/188 - Accuracy: 0.612, Train 226/381 - Accuracy: